In [ ]:
from time import sleep, time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
import sys
from BayesianOptimization_master.bayes_opt1 import BayesianOptimization, UtilityFunction
import statistics

    
def ackley(x, y):
    sleep(np.sqrt(x**2 + y**2) / 10)
    return -20 * np.exp(-0.2 * np.sqrt(0.5 * (x ** 2 + y ** 2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + np.e + 20

def rosenbrock(x, y):
    sleep(np.sqrt(x**2 + y**3) / 10)
    return 100 * (y - x ** 2) ** 2 + (1 - x) ** 2

def rastrigin(x, y):
    sleep((x + y**2 + x*y)/10)
    return 20 + x ** 2 - 10 * np.cos(2 * np.pi * x) + y ** 2 - 10 * np.cos(2 * np.pi * y)

    

In [ ]:
bounds = {'x': (-5, 5), 'y': (-5, 5)}
current_optimizing_fun = lambda x, y: -1 * ackley(x, y)

ei_max_points_time = [] 
ei_max_points = []       

eit_max_points_time = [] 
eit_max_points = []      

for i in range(3):
    ei_max_points_time.append([]) 
    ei_max_points.append([])        
    eit_max_points_time.append([]) 
    eit_max_points.append([])   

    eit_optimizer = BayesianOptimization(
        f=current_optimizing_fun,
        pbounds=bounds,
        time_travel=True
    )

    utility = UtilityFunction(kind="eit", kappa=2.5, xi=0.01)

    max_point = -1 * np.inf
    start_time = time()
    num_points_tested = 0
    while time() - start_time < 150:
        num_points_tested += 1
        next_point_to_probe = eit_optimizer.suggest(utility)
        before_t = time()
        result = current_optimizing_fun(**next_point_to_probe)
        delta_t = time() - before_t
        if result > max_point:
            max_point = result
            eit_max_points[i].append(-1 * max_point)
            eit_max_points_time[i].append(time() - start_time)
            #print(f"Maximum Found: {max_point}")
        eit_optimizer.register(params=next_point_to_probe, target=result,delta_t=delta_t)
    print(f"EIT Total Points Tested: {num_points_tested}")

    ei_optimizer = BayesianOptimization(
        f=current_optimizing_fun,
        pbounds=bounds,
        allow_duplicate_points=True,
        time_travel=True
    )

    utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.01)
    max_point = -1 * np.inf

    start_time = time()
    num_points_tested = 0
    while time() - start_time < 150:
        num_points_tested += 1
        next_point_to_probe = ei_optimizer.suggest(utility)
        before_t = time()
        result = current_optimizing_fun(**next_point_to_probe)
        delta_t = time() - before_t
        if result > max_point:
            max_point = result
            ei_max_points[i].append(-1 * max_point)
            ei_max_points_time[i].append(time() - start_time)
            #print(f"Maximum Found: {max_point}")
        ei_optimizer.register(params=next_point_to_probe, target=result,delta_t=delta_t)
    print(f" EI Total Points Tested: {num_points_tested}")

ei_avg_vals = []
ei_avg_times = []
eit_avg_vals = []
eit_avg_times = []
for i in range(100):
    to_avg_vals = []
    to_avg_times = []
    for j in range(len(ei_max_points)):
        if len(ei_max_points[j]) > i:
            to_avg_vals.append(ei_max_points[j][i])
            to_avg_times.append(ei_max_points_time[j][i])
    if len(to_avg_vals) > 0:
        ei_avg_times.append(statistics.mean(to_avg_times))
        #ei_avg_times.append(i)
        ei_avg_vals.append(statistics.mean(to_avg_vals))
    to_avg_vals = []
    to_avg_times = []
    for j in range(len(eit_max_points)):
        if len(eit_max_points[j]) > i:
            to_avg_vals.append(eit_max_points[j][i])
            to_avg_times.append(eit_max_points_time[j][i])
    if len(to_avg_vals) > 0:
        eit_avg_times.append(statistics.mean(to_avg_times))
        #eit_avg_times.append(i)
        eit_avg_vals.append(statistics.mean(to_avg_vals))

# Plot the first line
ei_avg_vals.sort(reverse=True)
ei_avg_times.sort()
plt.plot(ei_avg_times, ei_avg_vals, label='EI Line')

# Plot the second line
eit_avg_vals.sort(reverse=True)
eit_avg_times.sort()
plt.plot(eit_avg_times, eit_avg_vals, label='EIT Line')

# Add labels and title
plt.xlabel("Time (s)")
plt.ylabel("Minimum Value Found")
plt.title("Bayesian Optimization of Ackley Function Comparing EI and EIT, average of ten runs")

# Add legend
plt.legend()

# Show the plot
plt.show()



In [ ]:
import BayesOptTime

bounds = {'x': (-5, 5), 'y': (-5, 5)}
current_optimizing_fun = lambda x, y: -1 * ackley(x, y)

# utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)
max_point = -1 * np.inf
max_points = []
max_points_time = []
start_time = time()
num_points_tested = 0
while time() - start_time < 30:
    num_points_tested += 1
    next_point_to_probe = optimizer.suggest(utility)
    result = current_optimizing_fun(**next_point_to_probe)
    if result > max_point:
        max_point = result
        max_points.append(-1 * max_point)
        max_points_time.append(time() - start_time)
        print(f"Maximum Found: {max_point}")
    optimizer.register(params=next_point_to_probe, target=result)
print(f"Total Points Tested: {num_points_tested}")

random_guesses, fun_values, time_values = BayesOptTime(current_optimizing_fun, bounds, )

plt.plot(fun_values, time_values)
plt.title("BT Optimization of Ackley Function with EI Utility Function")
plt.xlabel("Time (s)")
plt.ylabel("Minimum Value Found")
plt.show()